Master model: a view that selects cleaned and renamed columns from main.cinema_tickets.

In [46]:
#I import duckdb to be able to use python and write queries
import duckdb 

In [47]:
#Connecting to the duckdb database that contains the data
conn = duckdb.connect('c:/Users/Fgllc/OneDrive/Escritorio/USC/etl_project/cinema_tickets.duckdb')
conn

In [48]:
# I'm checking which tables are included in the schema.
# 'cinema_tickets' was my original flat file that I imported.
# 'base_model' is the dbt model I created with all the cleaning
conn.execute('SHOW TABLES').fetchall()

[('base_model',),
 ('cinema_tickets',),
 ('fifth_model',),
 ('fourth_model',),
 ('master_model',),
 ('second_model',),
 ('seventh_model',),
 ('sixth_model',),
 ('third_model',)]

RAW TICKETING DATA: 


In [49]:
#CHECKING THE RAW TICKETING CSV DATA AS EXTRACTED FROM KAGGLE:
raw_data = """
SELECT *
FROM cinema_tickets
ORDER BY total_sales  DESC
"""
conn.execute(raw_data).fetchdf().head()

,film_code,cinema_code,total_sales,tickets_sold,tickets_out,show_time,occu_perc,ticket_price,ticket_use,capacity,date,month,quarter,day
0,1554,448,1262819994,8499,0,47,87.69,148584.538652,8499,9692.097160,2018-07-20,7,3,20
1,1554,448,1166354984,7889,0,43,88.19,147845.732539,7889,8945.458669,2018-07-13,7,3,13
2,1554,448,1058979995,7121,0,38,85.80,148712.258812,7121,8299.533800,2018-08-10,8,3,10
3,1554,448,1042239993,7020,0,39,84.73,148467.235470,7020,8285.141036,2018-07-27,7,3,27
4,1554,448,1003694994,6748,0,37,86.20,148739.625667,6748,7828.306265,2018-08-03,8,3,3


ISSUES WITH TOTAL_SALES COLUMN. IT LOOKS LIKE IS TOTAL_SALES IS, FOR SOME REASON, IN CENTS USD. 


TO FIX IT: I'M GOING TO DIVIDE BY 100.0 TO CONVERT TO USD TO THEN FINALLY HAVE TOTAL_REVENUE_USD.

In [50]:
source = """
SELECT
    film_code,
    cinema_code,
    ROUND(total_sales / 100.0, 2) AS total_revenue_usd,
    tickets_sold,
    tickets_out,
    show_time,
    occu_perc,
    ticket_use,
    capacity,
    date,
    month,
    quarter,
    day
FROM cinema_tickets
"""
conn.execute(source).fetchdf()

,film_code,cinema_code,total_revenue_usd,tickets_sold,tickets_out,show_time,occu_perc,ticket_use,capacity,date,month,quarter,day
0,1492,304,39000.0,26,0,4,4.26,26,610.328638,2018-05-05,5,2,5
1,1492,352,33600.0,42,0,5,8.08,42,519.801980,2018-05-05,5,2,5
2,1492,489,25600.0,32,0,4,20.00,32,160.000000,2018-05-05,5,2,5
3,1492,429,12000.0,12,0,1,11.01,12,108.991826,2018-05-05,5,2,5
4,1492,524,12000.0,15,0,3,16.67,15,89.982004,2018-05-05,5,2,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...
142519,1569,495,13200.0,22,0,2,3.86,22,569.948187,2018-11-04,11,4,4
142520,1569,474,12000.0,15,0,1,65.22,15,22.999080,2018-11-04,11,4,4
142521,1569,524,10600.0,8,0,3,9.20,8,86.956522,2018-11-04,11,4,4
142522,1569,529,6000.0,5,0,2,5.00,5,100.000000,2018-11-04,11,4,4


MASTER MODEL QUERY:

In [51]:
#Now, I use SQL within Python to output how "master_model" looks like
#Notice I created a query variable to write the SQL query the way I'm used to, clean and concise
query = """
SELECT *
FROM master_model
ORDER BY total_revenue_usd DESC
"""


conn.execute(query).fetchdf()

,film_code,cinema_code,total_revenue_usd,tickets_sold,tickets_out,show_time,occu_perc,ticket_use,capacity,date,month,quarter,day
0,1554,448,631410.00,8499,0,47,87.69,8499,9692.097160,2018-07-20,7,3,20
1,1554,448,583177.49,7889,0,43,88.19,7889,8945.458669,2018-07-13,7,3,13
2,1554,448,529490.00,7121,0,38,85.80,7121,8299.533800,2018-08-10,8,3,10
3,1554,448,521120.00,7020,0,39,84.73,7020,8285.141036,2018-07-27,7,3,27
4,1554,448,501847.50,6748,0,37,86.20,6748,7828.306265,2018-08-03,8,3,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...
142519,1511,532,14.00,4,0,2,0.67,4,597.014925,2018-07-07,7,3,7
142520,1493,214,12.50,1,0,3,0.10,1,1000.000000,2018-08-21,8,3,21
142521,1564,451,12.50,1,0,2,0.43,1,232.558140,2018-09-12,9,3,12
142522,1494,426,12.00,4,0,2,0.91,4,439.560440,2018-06-15,6,2,15


In [52]:
#This is just to close the connection in this notebook to be able to connect to the db in another notebook
conn.close()